### Continuous Bag of Words(CBOW)

Preprocess the Text

In [3]:
import numpy as np
from collections import Counter

In [13]:
text = "The cat sits on the mat. The cat plays with a ball."

# Tokenize the text
tokens = text.lower().split()
print(tokens)

# Create vocabulary
vocab = list(set(tokens))
print(vocab)

vocab_size = len(vocab)
# print(vocab_size)

# Map words to indices and vice versa
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
print(word_to_idx)
idx_to_word = {idx: word for idx, word in enumerate(vocab)}
print(idx_to_word)
# print("Vocabulary:", vocab)
# print("Word to Index:", word_to_idx)



['the', 'cat', 'sits', 'on', 'the', 'mat.', 'the', 'cat', 'plays', 'with', 'a', 'ball.']
['a', 'plays', 'on', 'with', 'the', 'ball.', 'mat.', 'sits', 'cat']
{'a': 0, 'plays': 1, 'on': 2, 'with': 3, 'the': 4, 'ball.': 5, 'mat.': 6, 'sits': 7, 'cat': 8}
{0: 'a', 1: 'plays', 2: 'on', 3: 'with', 4: 'the', 5: 'ball.', 6: 'mat.', 7: 'sits', 8: 'cat'}


Prepare Training Data

In [8]:
# Define context window size
window_size = 2

# Generate context-target pair
data = []
for i in range(window_size, len(tokens) - window_size):
    context = [tokens[i - j] for j in range(1, window_size + 1)]
    context += [tokens[i + j] for j in range(1, window_size + 1)]
    target = tokens[i]
    data.append((context, target))

print("Context-Target Pairs:", data)

Context-Target Pairs: [(['cat', 'the', 'on', 'the'], 'sits'), (['sits', 'cat', 'the', 'mat.'], 'on'), (['on', 'sits', 'mat.', 'the'], 'the'), (['the', 'on', 'the', 'cat'], 'mat.'), (['mat.', 'the', 'cat', 'plays'], 'the'), (['the', 'mat.', 'plays', 'with'], 'cat'), (['cat', 'the', 'with', 'a'], 'plays'), (['plays', 'cat', 'a', 'ball.'], 'with')]


Convert to One-Hot Encoding

In [9]:
# One-hot encode
def one_hot_encode(word, vocab_size):
    vector = np.zeros(vocab_size)
    vector[word_to_idx[word]] = 1
    return vector

# Prepare input and output for the neural network
X = []
y = []

for context, target in data:
    context_vectors = np.sum([one_hot_encode(word, vocab_size) for word in context], axis=0)
    X.append(context_vectors)
    y.append(one_hot_encode(target, vocab_size))

X = np.array(X)
y = np.array(y)

print("Input Shape (X):", X.shape)
print("Output Shape (y):", y.shape)


Input Shape (X): (8, 9)
Output Shape (y): (8, 9)


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(10, input_dim=vocab_size, activation='relu'),  # Hidden layer
    Dense(vocab_size, activation='softmax')             # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=500, verbose=1)


C:\Users\BJIT\Documents\Project\Word Embedding\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2500 - loss: 2.2119
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.2500 - loss: 2.2073
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.2500 - loss: 2.2028
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.2500 - loss: 2.1982
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.2500 - loss: 2.1937
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.2500 - loss: 2.1892
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.2500 - loss: 2.1846
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.2500 - loss: 2.1800
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.2500 - loss: 2.1755
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.2500 - loss: 2.1709
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.2500 - loss: 2.1664
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.

Test the Model

In [14]:
# Test the model
def predict_target(context_words):
    context_vector = np.sum([one_hot_encode(word, vocab_size) for word in context_words], axis=0)
    prediction = model.predict(context_vector.reshape(1, -1))
    predicted_word = idx_to_word[np.argmax(prediction)]
    return predicted_word

# Example test
test_context = ["the", "cat", "on", "the"]
predicted_word = predict_target(test_context)
print("Context Words:", test_context)
print("Predicted Target Word:", predicted_word)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Context Words: ['the', 'cat', 'on', 'the']
Predicted Target Word: mat.
